#### LangChain vs LangGraph (feat. LangGraph 개념 설명)
* LangGraph의 개념과 주요 기능을 이해하고, 차이점을 비교합니다.

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[30:])

gs
WD


In [3]:
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model='gpt-4o-mini') # 테스트의 경우에는 작은 모델을 사용합니다

from langchain_upstage import ChatUpstage
llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.5
    )
print(llm)

query = 'LangGraph는 무엇인가요?'
llm.invoke(query)



c:\Users\USER\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-CGunLKE6-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


client=<openai.resources.chat.completions.completions.Completions object at 0x0000025184FCE050> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000251854EA550> model_name='solar-pro' temperature=0.5 model_kwargs={} upstage_api_key=SecretStr('**********') upstage_api_base='https://api.upstage.ai/v1'


AIMessage(content='**LangGraph**는 **LangChain**과 통합된 그래프 기반의 상태 관리 프레임워크로, 복잡한 대화형 AI 애플리케이션(예: 챗봇, 에이전트, 멀티모달 시스템)을 구축할 때 **상태(state)와 흐름을 체계적으로 관리**하기 위해 설계되었습니다.  \n\n### 🔍 **핵심 개념**\n1. **그래프 구조**  \n   - 노드(Node)와 엣지(Edge)로 구성된 방향성 그래프로, 각 노드는 상태 전이(state transition) 또는 작업(예: LLM 호출, 데이터베이스 조회)을 나타냅니다.  \n   - 예: `대화 시작 → 사용자 입력 처리 → 외부 API 호출 → 응답 생성`과 같은 흐름을 시각적으로 표현.\n\n2. **상태 관리**  \n   - 대화 기록, 사용자 컨텍스트, 메모리 변수 등을 그래프 내에서 추적하며, 복잡한 세션에서도 일관된 상태를 유지합니다.  \n   - LangChain의 `ConversationBufferMemory` 등과 연동 가능.\n\n3. **확장성**  \n   - 커스텀 노드/엣지를 추가해 복잡한 로직(예: 분기 처리, 조건부 흐름)을 구현할 수 있습니다.  \n   - 예: 사용자 질문에 따라 다른 LLM 체인을 호출하는 분기 구조.\n\n4. **LangChain과의 통합**  \n   - LangChain의 체인(Chain), 도구(Tools), 메모리(Memory) 등을 LangGraph 노드에서 직접 활용 가능.\n\n---\n\n### 🛠 **사용 사례**\n- **멀티턴 챗봇**: 대화 이력을 유지하며 복잡한 응답 생성.  \n- **에이전트 시스템**: 외부 도구 호출 후 결과를 기반으로 동적 흐름 제어.  \n- **워크플로우 자동화**: 조건부 분기, 병렬 처리, 에러 핸들링 등.  \n\n---\n\n### 📦 **예시 코드 (간단한 그래프)**\n```python\nfrom langgraph import State, Graph\nfrom l

#### LangGraph의 기본개념
* `state`는 LangGraph 에이전트의 state를 나타내는 데이터 구조입니다.
* `state`는 `TypedDict`를 사용하여 정의되며, 이는 Python의 타입 힌팅을 통해 구조를 명확히 합니다.
    * 간단하게 `messages`라는 필드만 있습니다.
    * 필요에 따라 다양한 값들을 활용할 수 있습니다.
* `state`는 에이전트의 동작을 결정하는 데 사용되며, 각 노드에서 state를 업데이트하거나 참조할 수 있습니다.
* `state`는 LangGraph의 노드 간에 전달되며, 에이전트의 state 전이를 관리합니다.

In [4]:
from typing import Annotated # 타입 힌트를 사용하기 위해 
from typing_extensions import TypedDict # 구조화된 딕셔너리 타입을 정의하기 위해 

from langgraph.graph.message import add_messages 
from langchain_core.messages import AnyMessage # LangChain에서 사용하는 모든 종류의 메시지(예: HumanMessage, AIMessage)

# AgentState는 에이전트의 현재 상태를 나타내는 딕셔너리 타입을 정의합니다.
# TypedDict를 사용하면 딕셔너리가 어떤 키와 값 타입을 가져야 하는지 명확하게 지정할 수 있습니다.
class AgentState(TypedDict):
    # 'messages' 키는 에이전트의 대화 기록을 저장합니다.
    # 이 목록에는 LangChain 메시지 객체(AnyMessage)가 들어갑니다.
    # LangGraph가 이 상태를 처리할 때, 새로운 메시지가 추가되면
    # 기존 메시지 목록의 끝에 자동으로 추가되도록(append) 설정합니다.
    messages: list[Annotated[AnyMessage, add_messages]]

- 위에 선언한 `AgentState`를 활용하여 `StateGraph`를 생성합니다.

In [5]:
from langgraph.graph import StateGraph

graph_builder = StateGraph(AgentState)

- `graph`에 추가할 `node`를 생성합니다
-  `node`는 LangGraph에서 실행되는 개별적인 작업 단위를 의미합니다. 
    - 각 노드는 특정 기능을 수행하는 독립적인 컴포넌트로, 예를 들어 텍스트 생성, 데이터 처리, 또는 의사 결정과 같은 작업을 담당할 수 있습니다.
    - `node`는 기본적으로 함수(function)로 정의되고, 뒤에서 다루지만 다른 에이전트(agent)를 활용할 수도 있습니다

In [6]:
def generate(state: AgentState) -> AgentState:
    """
    `generate` 노드는 사용자의 질문을 받아서 응답을 생성하는 노드입니다.
    """
    messages = state['messages']
    ai_message = llm.invoke(messages)
    return {'messages': [ai_message]}

- `node`를 생성한 후에 `edge`로 연결합니다
- `edge`는 노드들 사이의 연결을 나타내며, 데이터와 제어 흐름의 경로를 정의합니다. 
    - 엣지를 통해 한 노드의 출력이 다음 노드의 입력으로 전달되어, 전체적인 워크플로우가 형성됩니다.
    - `node`와 `edge`의 조합은 방향성 그래프(Directed Graph)를 형성하며, 이를 통해 복잡한 AI 에이전트의 행동 흐름을 구조화할 수 있습니다

In [7]:
graph_builder.add_node('generate', generate)

- 모든 그래프는 `START(시작)`와 `END(종료)`가 있습니다
    - `END`를 explicit하게 선언하지 않는 경우도 종종 있지만, 가독성을 위해 작성해주는 것을 권장합니다

In [8]:
from langgraph.graph import START, END

graph_builder.add_edge(START, 'generate')
graph_builder.add_edge('generate', END)

- `node`를 생성하고 `edge`로 연결한 후에 `compile` 메서드를 호출하여 `Graph`를 생성합니다

In [9]:
graph = graph_builder.compile()
print(type(graph))

<class 'langgraph.graph.state.CompiledStateGraph'>


- `compile` 후에는 그래프를 시각화하여 확인할 수 있습니다
- 의도한대로 그래프가 생성됐는지 확인하는 습관을 기르는 것이 좋습니다
    - `git`에서 코드 작업물을 commit하기 전에 `git diff`를 통해 변경사항을 확인하는 것과 같습니다

In [ ]:
#from IPython.display import display, Image

#display(Image(graph.get_graph().draw_mermaid_png(max_retries=5, retry_delay=2.0)))

In [10]:
# 대체 방법
mermaid_code = graph.get_graph().draw_mermaid()
print("Mermaid Code:")
print(mermaid_code)

Mermaid Code:
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	generate(generate)
	__end__([<p>__end__</p>]):::last
	__start__ --> generate;
	generate --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



* https://mermaid.live/ 에서  mermain_code 로 직접 확인한다.
* [Graph 이미지](https://mermaidchart.com/play?utm_source=mermaid_live_editor&utm_medium=share#pako:eNpVkEFuwjAQRa9iTTcgJWlwSXANYlOO0FXrCplknFhKnMhxpFLE3WsMpMUbz9f89z3jExRdicChsrKvyftuLYxw-_3gpPXX7HPTbye1ee63X3POudJ2cBdjhQatdDi7F_Mrjqac4FBPaCOv5BRK4nhL7vj6f2jo3PjQKDw87FCREpUcG0eUbhr-pKhKlYoabTCuUVe144uEPgBh4GCPu14W2h15-mC4jHWLO6hDrgqI_JfoErizI0bQom3lRcJJGEIEuBpbFMB9eZtGgDBnj_XSfHRdeydtN1Y1cCWbwauxL_1mOy39f_9Z_Ipo37rROOCLVYgAfoJvrzKa5IzmKaUsY9lqmUdwBL6kyas_LH3JM7akjJ4j-AmPpglbZedfLmWdwA)


In [11]:
from langchain_core.messages import HumanMessage

initial_state = {'messages': [HumanMessage(query)]}
graph.invoke(initial_state)

{'messages': [AIMessage(content='**LangGraph**는 **대규모 언어 모델(LLM)**과 **상태 관리를 결합**한 **그래프 기반 프레임워크**로, 복잡한 LLM 애플리케이션을 구조화하고 확장 가능하게 설계할 수 있도록 도와줍니다. LangChain의 확장 프로젝트로 개발되었으며, 특히 **장기적인 대화 상태 추적**, **멀티모달 작업**, **복잡한 워크플로우 자동화**에 최적화되어 있습니다.\n\n### 📌 **LangGraph의 주요 특징**\n1. **그래프 기반 아키텍처**  \n   - 노드(LLM, 도구, 메모리 등)와 엣지(상태 전이)로 구성된 **유한 상태 머신(FSM)** 또는 **다이내믹 플로우**를 정의할 수 있습니다.\n   - 예: 사용자 입력 → LLM 처리 → 외부 API 호출 → 응답 생성 → 상태 업데이트.\n\n2. **상태(State) 관리**  \n   - 대화 기록, 변수, 컨텍스트를 그래프 실행 중에 유지하며 **장기적 맥락**을 보존합니다.\n   - `State` 객체를 통해 각 노드 간 데이터를 전달합니다.\n\n3. **LangChain과의 통합**  \n   - LangChain의 체인(Chains), 도구(Tools), 메모리(Memory) 컴포넌트를 LangGraph 노드로 쉽게 통합할 수 있습니다.\n\n4. **확장성**  \n   - 복잡한 워크플로우(예: 다단계 폼 채우기, 챗봇 대화 상태 관리)를 모듈식으로 구축할 수 있습니다.\n\n---\n\n### 🧩 **사용 사례 예시**\n- **멀티턴 챗봇**: 이전 대화를 기반으로 상태를 유지하며 응답합니다.  \n- **워크플로우 자동화**: LLM이 여러 도구와 상호작용하며 작업을 완료합니다(예: 예약 시스템).  \n- **에이전트 시스템**: 조건 분기와 상태 전이를 통해 동적 의사 결정을 구현합니다.\n\n---\n\n### 🛠 **간단한 예시 코드 (Python)**\n```python\nfrom langg